In [12]:
!pip install pandas selenium bs4 webdriver_manager
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata (12 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached charset_normalizer-3.4.1-cp313-cp313-macosx_10_13_universal2.whl.metadata (35 kB)
Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl (27 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.1-cp313-cp313-macosx_10_13_universal2.whl (195 kB)


In [13]:


options = webdriver.ChromeOptions()
options.add_argument("--headless")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

url = "https://www.fao.org/faostat/en/#data"
driver.get(url)

time.sleep(5)

html_content = driver.page_source

soup = BeautifulSoup(html_content, "html.parser")

subcategories = soup.find_all("li", class_="domain-list")

base_url = "https://www.fao.org/faostat/en/"

results = []
for item in subcategories:
    a_tag = item.find("a", class_="instafilta-target")
    if a_tag:
        subcategory_name = a_tag.get_text(strip=True)
        subcategory_link = a_tag.get("href")
        if subcategory_link.startswith("#"):
            full_link = base_url + subcategory_link
        else:
            full_link = subcategory_link
        results.append({"Subcategory": subcategory_name, "FullLink": full_link})

driver.quit()

df = pd.DataFrame(results)

In [19]:
df

,Subcategory,FullLink
0,Crops and livestock products,https://www.fao.org/faostat/en/#data/QCL
1,Production Indices,https://www.fao.org/faostat/en/#data/QI
2,Value of Agricultural Production,https://www.fao.org/faostat/en/#data/QV
3,Suite of Food Security Indicators,https://www.fao.org/faostat/en/#data/FS
4,Food Balances (2010-),https://www.fao.org/faostat/en/#data/FBS
...,...,...
62,Food Aid Shipments (WFP),https://www.fao.org/faostat/en/#data/FA
63,Machinery,https://www.fao.org/faostat/en/#data/RM
64,Machinery Archive,https://www.fao.org/faostat/en/#data/RY
65,Fertilizers archive,https://www.fao.org/faostat/en/#data/RA


In [20]:
import os
import time
import io
import requests
import zipfile
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


def download_and_extract_normalized_data(
    category_url, category_name, wait_time=5, max_retries=3
):
    """
    Downloads and extracts the "All Data Normalized" dataset for a given subcategory.
    Retries up to max_retries times if the download link isn't found or download fails.

    Parameters:
      category_url (str): The URL of the subcategory page.
      category_name (str): The name of the category. This will be used as the folder name.
      wait_time (int): Seconds to wait for dynamic content to load (default is 5).
      max_retries (int): Maximum number of attempts to retrieve and download the file.
    """
    normalized_link = None
    for attempt in range(1, max_retries + 1):
        print(f"Attempt {attempt} of {max_retries} to retrieve download link for {category_name}")
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)

        driver.get(category_url)
        time.sleep(wait_time) 
        page_html = driver.page_source
        driver.quit()


        soup = BeautifulSoup(page_html, "html.parser")
        bulk_section = soup.find(
            "div", class_="panel-footer", attrs={"data-role": "bulk-downloads-panel"}
        )
        if bulk_section:
            for a_tag in bulk_section.find_all("a"):
                if "All Data Normalized" in a_tag.get_text():
                    normalized_link = a_tag.get("data-bulkdownload")
                    break

        if normalized_link:
            print("Download link found:", normalized_link)
            break
        else:
            print(
                "Could not find the 'All Data Normalized' download link. Retrying in 3 seconds..."
            )
            time.sleep(3)

    if not normalized_link:
        print("Failed to retrieve the download link after maximum retries.")
        return

    folder_path = os.path.join("data", category_name)
    os.makedirs(folder_path, exist_ok=True)

    success = False
    for attempt in range(1, max_retries + 1):
        print(f"Attempt {attempt} of {max_retries} to download the file")
        response = requests.get(normalized_link)
        if response.status_code == 200:
            try:
                with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
                    zip_file.extractall(folder_path)
                print(f"Extracted files to folder: {folder_path}")
                success = True
                break
            except Exception as e:
                print("Error during extraction:", e)
        else:
            print("Failed to download the file. Status code:", response.status_code)

        if not success:
            print("Retrying download in 3 seconds...")
            time.sleep(3)

    if not success:
        print("Failed to download and extract the file after maximum retries.")

In [ ]:
from tqdm.auto import tqdm

for index, row in tqdm(df.iterrows(), total=len(df)):
    download_and_extract_normalized_data(row["FullLink"], row["Subcategory"])

/Users/shaoormunir/studies/sta-220/project-data-collection/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/67 [00:00<?, ?it/s]

Attempt 1 of 3 to retrieve download link for Crops and livestock products
Download link found: https://bulks-faostat.fao.org/production/Production_Crops_Livestock_E_All_Data_(Normalized).zip
Attempt 1 of 3 to download the file


  1%|▏         | 1/67 [00:08<09:14,  8.40s/it]

Extracted files to folder: data/Crops and livestock products
Attempt 1 of 3 to retrieve download link for Production Indices
Download link found: https://bulks-faostat.fao.org/production/Production_Indices_E_All_Data_(Normalized).zip
Attempt 1 of 3 to download the file


  3%|▎         | 2/67 [00:17<09:27,  8.73s/it]

...

Extracted files to folder: data/Machinery Archive
Attempt 1 of 3 to retrieve download link for Fertilizers archive
Download link found: https://bulks-faostat.fao.org/production/Inputs_FertilizersArchive_E_All_Data_(Normalized).zip
Attempt 1 of 3 to download the file


 99%|█████████▊| 66/67 [10:38<00:08,  8.30s/it]

Extracted files to folder: data/Fertilizers archive
Attempt 1 of 3 to retrieve download link for Producer Prices (old series)
Download link found: https://bulks-faostat.fao.org/production/PricesArchive_E_All_Data_(Normalized).zip
Attempt 1 of 3 to download the file


100%|██████████| 67/67 [10:47<00:00,  9.66s/it]

Extracted files to folder: data/Producer Prices (old series)
